In [1]:
from fractions import Fraction as fr

import pandas as pd
import numpy as np


class ConsistencyTest(object):
    """ 单排序和一致性检验 """    
    def __init__(self, A):
        self.RI = [0, 0, 0.58, 0.90, 1.12, 1.24, 1.32, 1.41, 1.45, 1.49, 1.51]
        self.A = A
        self.index = self.A.index.tolist()
        # 列向量求和，修改传入的A比较矩阵
        self.sum_a0 = []
        for i in A.index:
            self.sum_a0.append(np.sum(A[i]))
        self.A.loc['合计'] = self.sum_a0
        
    @property
    def eigenvector_max(self):
        """最大特征向量简化计算"""
        # 1.列向量归一化
        index_to_one = self.A[0:-1].apply(lambda x: x / self.A.loc['合计'], axis=1)
        # 2.求行和
        sum_col = np.sum(index_to_one, axis=1)
        # 3.归一化
        w = sum_col.apply(lambda x: x / np.sum(sum_col))
        return w

    @property
    def eigenvalue_max(self):
        """最大特征值，简便算法"""
        # 1.求Aw
        Aw = np.dot(self.A[0:-1],self.eigenvector_max)
        #Aw = pd.Series(Aw,index=self.index)
        # 2.求最大特征向量
        r = np.sum(Aw / self.eigenvector_max) / len(self.index)
        return r
    
    @property
    def consistency(self):
        """ 一致性检验 """
        CI= (self.eigenvalue_max - len(self.index)) / (len(self.index) - 1)
#         CI = (self.eigenvalue_max-4)/3
        # CR = CI/RI
        CR = CI / self.RI[len(self.index)-1]
        #CR = CI / 0.9
        if CR < 0.1:
            print('通过一致性检验!')
        else:
            print('没有通过!')
        consistency = {}
        consistency['CI'] = CI
        consistency['RI'] = self.RI[len(self.index)-1]
        consistency['CR'] = CR
        # todo 一致性检验
        print(consistency)
        return consistency



third = []   
def task_son(compare_list,index ):
    """传入数据列表，转换二维数组, 实例化,添加到third数组里面"""
    na = np.array(compare_list).reshape(4,4)
    df = pd.DataFrame(na,index=index,columns=index)
    global third
    B_3 = ConsistencyTest(df)
    third.append(B_3)
    return third

def consistency_all(third):
    """ 整体一致性检验 """
    CI = 0
    RI = 0
    for i in range(len(third)):
        print('\n第{}次子方案检验:'.format(i+1))
        consistency = third[i].consistency
        CI += A.eigenvector_max.iloc[i] * consistency['CI']
        RI += A.eigenvector_max.iloc[i] * consistency['RI']
    CR = CI / RI
    print('\n总排序一致性检验:')
    if CR < 0.1:
        print('通过一次性检验')
    else:
        print('没有通过一次性检验')
    print('CR:',CR)    
        
def order(third,index):
    # 求总排序
    # 1.求第三层方向向量
    w_3 = pd.DataFrame()
    for i in range(len(third)):
        w_3[index[i]] = third[i].eigenvector_max
     # 2.求总排序   
    sl = pd.Series(np.dot(w_3, A.eigenvector_max),index=index) 
    return sl

In [2]:
# 1. 准则层比较矩阵
# 1.1 构造
index = ['品牌信誉','功能','价格','电量']
A = pd.DataFrame([
    [1,1/2,1/3,4],
    [2,1,1/2,6],
    [3,2,1,9],
    [1/4,1/6,1/9,1]
],index=index,columns=index)
A = ConsistencyTest(A)
A.consistency
print('____________________________________________________________________')

通过一致性检验!
{'CI': 0.006920579354448897, 'RI': 0.9, 'CR': 0.00768953261605433}
____________________________________________________________________


In [3]:
# 2.方案层一致性检验
index=['苹果','华为','VIVO','小米']
#  苹果
B1_3 = [1,2,3,5,fr(1,2),1,2,3,fr(1,5),fr(1,2),1,2,fr(1,7),fr(1,3),fr(1,2),1]
task_son(B1_3,index)
# 功能
B2_3 = [1,3,1,3,fr(1,3),1,fr(1,3),1,1,3,1,3,fr(1,3),1,fr(1,3),1]
task_son(B2_3,index)
# 价格
B3_3 = [1,1/2,1/3,1/5,2,1,1/2,1/3,3,2,1,1/2,5,3,2,1]
task_son(B3_3,index)
# 电量
B4_3 = [1,2,1/2,1,1/2,1,1/3,1/2,2,3,1,2,1,2,1/2,1]
task_son(B4_3,index)
print('一致性检验:')
consistency_all(third)

一致性检验:

第1次子方案检验:
通过一致性检验!
{'CI': -0.05849875356216986, 'RI': 0.9, 'CR': -0.06499861506907761}

第2次子方案检验:
通过一致性检验!
{'CI': 0.0, 'RI': 0.9, 'CR': 0.0}

第3次子方案检验:
通过一致性检验!
{'CI': 0.004845243943055226, 'RI': 0.9, 'CR': 0.005383604381172473}

第4次子方案检验:
通过一致性检验!
{'CI': 0.0034551189065735364, 'RI': 0.9, 'CR': 0.0038390210073039293}

总排序一致性检验:
通过一次性检验
CR: -0.007988552201539677


In [4]:
#  4总排序
order(third,index)

苹果      0.245315
华为      0.166620
VIVO    0.287358
小米      0.300708
dtype: float64

In [5]:
A.A

,品牌信誉,功能,价格,电量
品牌信誉,1.00,0.500000,0.333333,4.0
功能,2.00,1.000000,0.500000,6.0
价格,3.00,2.000000,1.000000,9.0
电量,0.25,0.166667,0.111111,1.0
合计,6.25,3.666667,1.944444,20.0


In [6]:
third[0].A

,苹果,华为,VIVO,小米
苹果,1,2,3,5
华为,1/2,1,2,3
VIVO,1/5,1/2,1,2
小米,1/7,1/3,1/2,1
合计,129/70,23/6,13/2,11


In [7]:
third[1].A

,苹果,华为,VIVO,小米
苹果,1,3,1,3
华为,1/3,1,1/3,1
VIVO,1,3,1,3
小米,1/3,1,1/3,1
合计,8/3,8,8/3,8


In [8]:
third[2].A

,苹果,华为,VIVO,小米
苹果,1.0,0.5,0.333333,0.200000
华为,2.0,1.0,0.500000,0.333333
VIVO,3.0,2.0,1.000000,0.500000
小米,5.0,3.0,2.000000,1.000000
合计,11.0,6.5,3.833333,2.033333


In [9]:
third[3].A

,苹果,华为,VIVO,小米
苹果,1.0,2.0,0.500000,1.0
华为,0.5,1.0,0.333333,0.5
VIVO,2.0,3.0,1.000000,2.0
小米,1.0,2.0,0.500000,1.0
合计,4.5,8.0,2.333333,4.5
